<a href="https://colab.research.google.com/github/amiranissian/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/train1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [2]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [3]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [4]:
def get_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [5]:
batch_size = 128
epochs = 2

In [6]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1)

Epoch 1/2
422/422 [==============================] - 46s 107ms/step - loss: 0.6240 - accuracy: 0.7775 - val_loss: 0.4456 - val_accuracy: 0.8415
Epoch 2/2
422/422 [==============================] - 45s 106ms/step - loss: 0.3928 - accuracy: 0.8625 - val_loss: 0.3762 - val_accuracy: 0.8675


In [8]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.4 MB/s 
     |████████████████████████████████| 182 kB 50.8 MB/s 
     |████████████████████████████████| 168 kB 67.5 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 49.1 MB/s 
     |████████████████████████████████| 166 kB 68.3 MB/s 
     |████████████████████████████████| 166 kB 63.5 MB/s 
     |████████████████████████████████| 162 kB 66.3 MB/s 
     |████████████████████████████████| 162 kB 65.6 MB/s 
     |████████████████████████████████| 158 kB 51.7 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 52.5 MB/s 
     |████████████████████████████████| 157 kB 46.5 MB/s 
     |████████████████████████████████| 157 kB 48.6 MB/s 
     |████████████████████████████████| 157 kB 51.6 MB/s 
     |████████████████████████████

In [9]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [24]:
batch_size = 128
epochs = 6
wandb.init(project="övning3",
           config={"batch_size": batch_size, "epochs": epochs})

In [25]:
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])
wandb.finish()

Epoch 1/6
422/422 [==============================] - ETA: 0s - loss: 0.1221 - accuracy: 0.9551

wandb: Adding directory to artifact (/content/wandb/run-20221209_203334-3cnyto16/files/model-best)... Done. 0.0s


422/422 [==============================] - 43s 103ms/step - loss: 0.1221 - accuracy: 0.9551 - val_loss: 0.3108 - val_accuracy: 0.9057
Epoch 2/6
422/422 [==============================] - ETA: 0s - loss: 0.1191 - accuracy: 0.9568

wandb: Adding directory to artifact (/content/wandb/run-20221209_203334-3cnyto16/files/model-best)... Done. 0.0s


422/422 [==============================] - 44s 105ms/step - loss: 0.1191 - accuracy: 0.9568 - val_loss: 0.2883 - val_accuracy: 0.9105
Epoch 3/6
422/422 [==============================] - ETA: 0s - loss: 0.1136 - accuracy: 0.9591

wandb: Adding directory to artifact (/content/wandb/run-20221209_203334-3cnyto16/files/model-best)... Done. 0.0s


422/422 [==============================] - 43s 102ms/step - loss: 0.1136 - accuracy: 0.9591 - val_loss: 0.2874 - val_accuracy: 0.9097
Epoch 4/6
422/422 [==============================] - 48s 113ms/step - loss: 0.1113 - accuracy: 0.9606 - val_loss: 0.3042 - val_accuracy: 0.9050
Epoch 5/6
422/422 [==============================] - 42s 100ms/step - loss: 0.1096 - accuracy: 0.9607 - val_loss: 0.2960 - val_accuracy: 0.9098
Epoch 6/6
422/422 [==============================] - 43s 101ms/step - loss: 0.1039 - accuracy: 0.9626 - val_loss: 0.2928 - val_accuracy: 0.9080


accuracy,▁▃▅▆▆█
epoch,▁▂▄▅▇█
loss,█▇▅▄▃▁
val_accuracy,▂█▇▁▇▅
val_loss,█▁▁▆▄▃
accuracy,0.96263
best_epoch,2
best_val_loss,0.28745
epoch,5
loss,0.10388
val_accuracy,0.908
